In [1]:
import pandas as pd
import os
from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.manifold import TSNE
from collections import Counter
from six.moves import cPickle
import gensim.models.word2vec as w2v
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
import io
import re
import json
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [2]:
# http://help.sentiment140.com/for-students/
testing_data_set = pd.read_csv(r"C:\Users\jjtre\Documents\UCSD Coding Bootcamp Data Science\Final Project\raw_data\testdata.manual.2009.06.14.csv", encoding='latin-1', header = None)
testing_data_set.head()

,0,1,2,3,4,5
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...


In [3]:
#  the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
#training_data_set.drop(training_data_set.columns[1:5], axis=1, inplace=True)
testing_data_set.drop(testing_data_set.columns[1:5], axis=1, inplace=True)
testing_data_set.head()

,0,5
0,4,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,Reading my kindle2... Love it... Lee childs i...
2,4,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,@kenburbary You'll love your Kindle2. I've had...
4,4,@mikefish Fair enough. But i have the Kindle2...


In [5]:
len(testing_data_set)

498

In [6]:
testing_data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
0    498 non-null int64
5    498 non-null object
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [7]:
# check for missing records
pd.concat([testing_data_set.isnull().sum(), 100 * testing_data_set.isnull().sum()/len(testing_data_set)], 
              axis=1).rename(columns={0:'Missing Records', 1:'Percentage (%)'})

,Missing Records,Percentage (%)
0,0,0.0
5,0,0.0


In [8]:
#  the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
testing_data_set.rename(columns={0: 'tweet_polarity', 5 : 'tweet_text'}, inplace=True)
testing_data_set.head()

,tweet_polarity,tweet_text
0,4,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,Reading my kindle2... Love it... Lee childs i...
2,4,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,@kenburbary You'll love your Kindle2. I've had...
4,4,@mikefish Fair enough. But i have the Kindle2...


In [9]:
# no neutral tweets!?
testing_data_set['tweet_polarity'].value_counts()

4    182
0    177
2    139
Name: tweet_polarity, dtype: int64

In [10]:
def getCleanTwit(str1):
   words= str1.split()
   clean_words = []
   for i in words:
       if re.findall(r'^http', i):
           continue
       else:
           clean_words.append(i)
   text = u' '.join(x for x in clean_words)
   text = re.sub("http*:\/\/[0-9a-zA-Z\-\_]+\.[\-\_0-9a-zA-Z]", u"", text)
   text = re.sub("\@[\_0-9a-zA-Z]+\:?", u"", text)
   text = re.sub("\&amp\;?", u"", text)
   text = re.sub("[\:\.]{1,}$", u"", text)
   text = re.sub("^RT\:?", u"", text)
   return text

In [11]:
tweets = testing_data_set["tweet_text"].to_list()
tweets

['@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.',
 'Reading my kindle2...  Love it... Lee childs is good read.',
 'Ok, first assesment of the #kindle2 ...it fucking rocks!!!',
 "@kenburbary You'll love your Kindle2. I've had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)",
 "@mikefish  Fair enough. But i have the Kindle2 and I think it's perfect  :)",
 "@richardebaker no. it is too big. I'm quite happy with the Kindle2.",
 'Fuck this economy. I hate aig and their non loan given asses.',
 'Jquery is my new best friend.',
 'Loves twitter',
 'how can you not love Obama? he makes jokes about himself.',
 "Check this video out -- President Obama at the White House Correspondents' Dinner http://bit.ly/IMXUM",
 "@Karoli I firmly believe that Obama/Pelosi have ZERO desire to be civil.  It's a charade and a slogan, but they want to destroy conservatism",
 'House Correspondents dinner 

In [12]:
clean_tweets = []
for i in tweets:
   twi = getCleanTwit(i)
   clean_tweets.append(twi)
clean_tweets

[' I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right',
 'Reading my kindle2... Love it... Lee childs is good read',
 'Ok, first assesment of the #kindle2 ...it fucking rocks!!!',
 " You'll love your Kindle2. I've had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)",
 " Fair enough. But i have the Kindle2 and I think it's perfect :)",
 " no. it is too big. I'm quite happy with the Kindle2",
 'Fuck this economy. I hate aig and their non loan given asses',
 'Jquery is my new best friend',
 'Loves twitter',
 'how can you not love Obama? he makes jokes about himself',
 "Check this video out -- President Obama at the White House Correspondents' Dinner",
 " I firmly believe that Obama/Pelosi have ZERO desire to be civil. It's a charade and a slogan, but they want to destroy conservatism",
 'House Correspondents dinner was last night whoopi, barbara  sherri went, Obama got a standing ovation',
 'Watch

In [13]:
testing_data_set['clean_tweets'] = clean_tweets
testing_data_set.head()

,tweet_polarity,tweet_text,clean_tweets
0,4,@stellargirl I loooooooovvvvvveee my Kindle2. ...,I loooooooovvvvvveee my Kindle2. Not that the...
1,4,Reading my kindle2... Love it... Lee childs i...,Reading my kindle2... Love it... Lee childs is...
2,4,"Ok, first assesment of the #kindle2 ...it fuck...","Ok, first assesment of the #kindle2 ...it fuck..."
3,4,@kenburbary You'll love your Kindle2. I've had...,You'll love your Kindle2. I've had mine for a...
4,4,@mikefish Fair enough. But i have the Kindle2...,Fair enough. But i have the Kindle2 and I thi...


In [14]:
testing_data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 3 columns):
tweet_polarity    498 non-null int64
tweet_text        498 non-null object
clean_tweets      498 non-null object
dtypes: int64(1), object(2)
memory usage: 11.8+ KB


In [15]:
tokenizer = RegexpTokenizer(r'\w+')

In [16]:
testing_data_set['clean_tweets'] = testing_data_set['clean_tweets'].apply(lambda x: tokenizer.tokenize(x.lower()))
testing_data_set['clean_tweets'].head()

0    [i, loooooooovvvvvveee, my, kindle2, not, that...
1    [reading, my, kindle2, love, it, lee, childs, ...
2    [ok, first, assesment, of, the, kindle2, it, f...
3    [you, ll, love, your, kindle2, i, ve, had, min...
4    [fair, enough, but, i, have, the, kindle2, and...
Name: clean_tweets, dtype: object

In [17]:
testing_data_set.head(10)

,tweet_polarity,tweet_text,clean_tweets
0,4,@stellargirl I loooooooovvvvvveee my Kindle2. ...,"[i, loooooooovvvvvveee, my, kindle2, not, that..."
1,4,Reading my kindle2... Love it... Lee childs i...,"[reading, my, kindle2, love, it, lee, childs, ..."
2,4,"Ok, first assesment of the #kindle2 ...it fuck...","[ok, first, assesment, of, the, kindle2, it, f..."
3,4,@kenburbary You'll love your Kindle2. I've had...,"[you, ll, love, your, kindle2, i, ve, had, min..."
4,4,@mikefish Fair enough. But i have the Kindle2...,"[fair, enough, but, i, have, the, kindle2, and..."
5,4,@richardebaker no. it is too big. I'm quite ha...,"[no, it, is, too, big, i, m, quite, happy, wit..."
6,0,Fuck this economy. I hate aig and their non lo...,"[fuck, this, economy, i, hate, aig, and, their..."
7,4,Jquery is my new best friend.,"[jquery, is, my, new, best, friend]"
8,4,Loves twitter,"[loves, twitter]"
9,4,how can you not love Obama? he makes jokes abo...,"[how, can, you, not, love, obama, he, makes, j..."


In [18]:
# remove stop words 
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jjtre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
testing_data_set['clean_tweets'] = testing_data_set['clean_tweets'].apply(lambda x: remove_stopwords(x))
testing_data_set['clean_tweets'].head()

0    [loooooooovvvvvveee, kindle2, dx, cool, 2, fan...
1    [reading, kindle2, love, lee, childs, good, read]
2      [ok, first, assesment, kindle2, fucking, rocks]
3    [love, kindle2, mine, months, never, looked, b...
4              [fair, enough, kindle2, think, perfect]
Name: clean_tweets, dtype: object

In [21]:
# save option with token and stop words 
testing_data_set.to_csv("testing_cleaned_tweets_token_stopwords.csv", encoding='utf-8', index=False)

In [22]:
testing_data_set.head(10)

,tweet_polarity,tweet_text,clean_tweets
0,4,@stellargirl I loooooooovvvvvveee my Kindle2. ...,"[loooooooovvvvvveee, kindle2, dx, cool, 2, fan..."
1,4,Reading my kindle2... Love it... Lee childs i...,"[reading, kindle2, love, lee, childs, good, read]"
2,4,"Ok, first assesment of the #kindle2 ...it fuck...","[ok, first, assesment, kindle2, fucking, rocks]"
3,4,@kenburbary You'll love your Kindle2. I've had...,"[love, kindle2, mine, months, never, looked, b..."
4,4,@mikefish Fair enough. But i have the Kindle2...,"[fair, enough, kindle2, think, perfect]"
5,4,@richardebaker no. it is too big. I'm quite ha...,"[big, quite, happy, kindle2]"
6,0,Fuck this economy. I hate aig and their non lo...,"[fuck, economy, hate, aig, non, loan, given, a..."
7,4,Jquery is my new best friend.,"[jquery, new, best, friend]"
8,4,Loves twitter,"[loves, twitter]"
9,4,how can you not love Obama? he makes jokes abo...,"[love, obama, makes, jokes]"


In [23]:
# Instantiate lemmatizer
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

In [24]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jjtre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
testing_data_set['clean_tweets'] = testing_data_set['clean_tweets'].apply(lambda x: word_lemmatizer(x))
testing_data_set['clean_tweets'].head()

0    [loooooooovvvvvveee, kindle2, dx, cool, 2, fan...
1     [reading, kindle2, love, lee, child, good, read]
2       [ok, first, assesment, kindle2, fucking, rock]
3    [love, kindle2, mine, month, never, looked, ba...
4              [fair, enough, kindle2, think, perfect]
Name: clean_tweets, dtype: object

In [26]:
testing_data_set.head(10)

,tweet_polarity,tweet_text,clean_tweets
0,4,@stellargirl I loooooooovvvvvveee my Kindle2. ...,"[loooooooovvvvvveee, kindle2, dx, cool, 2, fan..."
1,4,Reading my kindle2... Love it... Lee childs i...,"[reading, kindle2, love, lee, child, good, read]"
2,4,"Ok, first assesment of the #kindle2 ...it fuck...","[ok, first, assesment, kindle2, fucking, rock]"
3,4,@kenburbary You'll love your Kindle2. I've had...,"[love, kindle2, mine, month, never, looked, ba..."
4,4,@mikefish Fair enough. But i have the Kindle2...,"[fair, enough, kindle2, think, perfect]"
5,4,@richardebaker no. it is too big. I'm quite ha...,"[big, quite, happy, kindle2]"
6,0,Fuck this economy. I hate aig and their non lo...,"[fuck, economy, hate, aig, non, loan, given, ass]"
7,4,Jquery is my new best friend.,"[jquery, new, best, friend]"
8,4,Loves twitter,"[love, twitter]"
9,4,how can you not love Obama? he makes jokes abo...,"[love, obama, make, joke]"


In [27]:
# save option with lemmatizing (maps to one base,  e.g. dogs to dog)
testing_data_set.to_csv("testing_cleaned_tweets_lemm.csv", encoding='utf-8', index=False)

In [28]:
# instantiate stemmer
stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = [stemmer.stem(i) for i in text]
    return stem_text

In [29]:
testing_data_set['clean_tweets'] = testing_data_set['clean_tweets'].apply(lambda x: word_stemmer(x))
testing_data_set['clean_tweets'].head()

0    [loooooooovvvvvvee, kindle2, dx, cool, 2, fant...
1        [read, kindle2, love, lee, child, good, read]
2              [ok, first, asses, kindle2, fuck, rock]
3    [love, kindle2, mine, month, never, look, back...
4              [fair, enough, kindle2, think, perfect]
Name: clean_tweets, dtype: object

In [30]:
# save option with stemming (more aggressive cuts prefixes and/or endings of words based on common ones)
testing_data_set.to_csv("testing_cleaned_tweets_stemmed.csv", encoding='utf-8', index=False)